# 1.chABSAデータセットを読み込み、DataLoaderの作成(BertのTokenizerを利用）

In [ ]:
# パスの追加
import sys
sys.path.append('/home/siny/miniconda3/envs/pytorch/lib/python36.zip')
sys.path.append('/home/siny/miniconda3/envs/pytorch/lib/python3.6')
sys.path.append('/home/siny/miniconda3/envs/pytorch/lib/python3.6/lib-dynload')
sys.path.append('/home/siny/.local/lib/python3.6/site-packages')
sys.path.append('/home/siny/miniconda3/envs/pytorch/lib/python3.6/site-packages')

In [ ]:
import random
import time
import numpy as np
from tqdm import tqdm
import torch 
from torch import nn
import torch.optim as optim
import torchtext

In [ ]:
from utils.dataloader import get_chABSA_DataLoaders_and_TEXT
from utils.bert import BertTokenizer


In [ ]:
train_dl, val_dl, TEXT, dataloaders_dict= get_chABSA_DataLoaders_and_TEXT(max_length=256, batch_size=32)

In [ ]:
# 動作確認 検証データのデータセットで確認
batch = next(iter(train_dl))
print("Textの形状=", batch.Text[0].shape)
print("Labelの形状=", batch.Label.shape)
print(batch.Text)
print(batch.Label)

In [ ]:
# ミニバッチの1文目を確認してみる
tokenizer_bert = BertTokenizer(vocab_file="./vocab/vocab.txt", do_lower_case=False)
text_minibatch_1 = (batch.Text[0][1]).numpy()

# IDを単語に戻す
text = tokenizer_bert.convert_ids_to_tokens(text_minibatch_1)

print(text)


# 2.BERTによるネガポジ分類モデル実装

In [ ]:
from utils.bert import get_config, BertModel,BertForchABSA, set_learned_params

# モデル設定のJOSNファイルをオブジェクト変数として読み込みます
config = get_config(file_path="./weights/bert_config.json")

# BERTモデルを作成します
net_bert = BertModel(config)

# BERTモデルに学習済みパラメータセットします
net_bert = set_learned_params(
    net_bert, weights_path="./weights/pytorch_model.bin")

In [ ]:
# モデル構築
net = BertForchABSA(net_bert)

# 訓練モードに設定
net.train()

print('ネットワーク設定完了')

# 3.BERTのファインチューニングに向けた設定

In [ ]:
# 勾配計算を最後のBertLayerモジュールと追加した分類アダプターのみ実行

# 1. まず全部を、勾配計算Falseにしてしまう
for name, param in net.named_parameters():
    param.requires_grad = False

# 2. 最後のBertLayerモジュールを勾配計算ありに変更
for name, param in net.bert.encoder.layer[-1].named_parameters():
    param.requires_grad = True

# 3. 識別器を勾配計算ありに変更
for name, param in net.cls.named_parameters():
    param.requires_grad = True


In [ ]:
# 最適化手法の設定

# BERTの元の部分はファインチューニング
optimizer = optim.Adam([
    {'params': net.bert.encoder.layer[-1].parameters(), 'lr': 5e-5},
    {'params': net.cls.parameters(), 'lr': 5e-5}
], betas=(0.9, 0.999))

# 損失関数の設定
criterion = nn.CrossEntropyLoss()
# nn.LogSoftmax()を計算してからnn.NLLLoss(negative log likelihood loss)を計算


In [ ]:
# 学習・検証を実施
from utils.train import train_model

# 学習・検証を実行する。
num_epochs = 1
net_trained = train_model(net, dataloaders_dict,
                          criterion, optimizer, num_epochs=num_epochs)


In [ ]:
# 学習したネットワークパラメータを保存します
save_path = './weights/bert_fine_tuning_chABSA_22epoch.pth'
torch.save(net_trained.state_dict(), save_path)


In [ ]:
# モデルの生成
net_trained = BertForchABSA(net_bert)
save_path = './weights/bert_fine_tuning_chABSA_22epoch.pth'
# 学習したネットワークパラメータをロード
net_trained.load_state_dict(torch.load(save_path, map_location='cpu'))
net_trained.eval()

# 4.サンプルの文章で推論とAttentionを可視化する。

In [4]:
from utils.config import *
from utils.predict import predict, create_vocab_text, build_bert_model
from IPython.display import HTML, display

In [ ]:
#TEXTオブジェクト（torchtext.data.field.Field）をpklファイルにダンプしておく（推論時に利用するため）
# 1度生成すればＯＫ
TEXT = create_vocab_text()

In [5]:
input_text = "以上の結果、当連結会計年度における売上高1,785百万円(前年同期比357百万円減、16.7％減)、営業損失117百万円(前年同期比174百万円減、前年同期　営業利益57百万円)、経常損失112百万円(前年同期比183百万円減、前年同期　経常利益71百万円)、親会社株主に帰属する当期純損失58百万円(前年同期比116百万円減、前年同期　親会社株主に帰属する当期純利益57百万円)となりました"
net_trained = build_bert_model()
html_output = predict(input_text, net_trained)
print("======================推論結果の表示======================")
print(input_text)
display(HTML(html_output))

['以上', 'の', '結果', '、', '当', '連結', '会計', '年度', 'に', 'おける', '売上高', '[UNK]', '，', '[UNK]', '円', '（', '前年', '同期', '比', '[UNK]', '円', '減', '、', '[UNK]', '．', '[UNK]', '％', '減', '）', '、', '営業', '損失', '[UNK]', '円', '（', '前年', '同期', '比', '[UNK]', '円', '減', '、', '前年', '同期', '営業', '利益', '[UNK]', '円', '）', '、', '[UNK]', '損失', '[UNK]', '円', '（', '前年', '同期', '比', '[UNK]', '円', '減', '、', '前年', '同期', '[UNK]', '利益', '[UNK]', '円', '）', '、', '親会社', '株主', 'に', '帰属', 'する', '[UNK]', '純', '損失', '[UNK]', '円', '（', '前年', '同期', '比', '[UNK]', '円', '減', '、', '前年', '同期', '親会社', '株主', 'に', '帰属', 'する', '[UNK]', '純', '利益', '[UNK]', '円', '）', 'と', 'なり', 'ました']
[2, 269, 5, 337, 6, 719, 3700, 5481, 594, 8, 217, 16720, 1, 176, 1, 387, 16, 2307, 3704, 2460, 1, 387, 4265, 6, 1, 264, 1, 257, 4265, 17, 6, 911, 7429, 1, 387, 16, 2307, 3704, 2460, 1, 387, 4265, 6, 2307, 3704, 911, 3718, 1, 387, 17, 6, 1, 7429, 1, 387, 16, 2307, 3704, 2460, 1, 387, 4265, 6, 2307, 3704, 1, 3718, 1, 387, 17, 6, 11100, 6970, 8, 13937, 22, 1, 3962